In [1]:
import requests 

url = "http://api.citybik.es/v2/networks/bicicorunha"
response = requests.get(url)
response.json()

{'network': {'id': 'bicicorunha',
  'name': 'Bicicoruña',
  'location': {'latitude': 43.3623,
   'longitude': -8.4115,
   'city': 'A Coruña',
   'country': 'ES'},
  'href': '/v2/networks/bicicorunha',
  'company': ['PBSC Urban Solutions'],
  'gbfs_href': 'https://acoruna.publicbikesystem.net/customer/gbfs/v2/gbfs.json',
  'stations': [{'id': '023efce1bbb332a1b918d56aeb671890',
    'name': 'Avenida de Arteixo',
    'latitude': 43.360755,
    'longitude': -8.410942,
    'timestamp': '2024-12-12T20:26:46.861693Z',
    'free_bikes': 2,
    'empty_slots': 13,
    'extra': {'uid': '53',
     'renting': True,
     'returning': True,
     'last_updated': 1734035023,
     'address': 'Avenida de Arteixo,29',
     'post_code': '15007',
     'payment': ['key', 'transitcard', 'creditcard', 'phone'],
     'payment-terminal': True,
     'altitude': 0.0,
     'slots': 15,
     'normal_bikes': 1,
     'ebikes': 1,
     'has_ebikes': True,
     'rental_uris': {}}},
   {'id': '02cecd02915c86d7ab8034b61b1

In [4]:
import requests
import os
import time
from pymongo import MongoClient

# Configuración desde variables de entorno
mongo_uri = os.getenv('MONGO_URI', 'mongodb://localhost:27017/')
db_name = os.getenv('DB_NAME', 'bicis_db')
collection_name = os.getenv('COLLECTION_NAME', 'bicicorunha')

# URL de la API
url = "http://api.citybik.es/v2/networks/bicicorunha"
intervalo_minutos = 2  

# Conexión a MongoDB
try:
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]
except Exception as e:
    print(f"Error al conectar a MongoDB: {e}")
    exit(1)

# Función para hacer la solicitud a la API
def obtener_datos():
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            stations = data["network"]["stations"]
            
            # Insertar todas las estaciones a la vez
            collection.insert_many(stations)
            print(f"{len(stations)} estaciones almacenadas correctamente.")
        else:
            error_message = f"Error {response.status_code}: {response.text}"
            print(error_message)
            raise Exception(error_message)  # Lanzar una excepción para detener el bucle
    except Exception as e:
        print(f"Error al conectar a la API o almacenar datos en MongoDB: {e}")
        raise  # Volver a lanzar la excepción para manejarla en el nivel superior

# Bucle principal para ejecutar la solicitud periódicamente
try:
    while True:
        obtener_datos()
        time.sleep(intervalo_minutos * 60)  
except KeyboardInterrupt:
    print("\nEjecución interrumpida por el usuario. Finalizando el programa...")
except Exception as e:
    print(f"Ejecución detenida debido a un error{e}")
finally:
    client.close()
    print("Conexión a MongoDB cerrada.")


49 estaciones almacenadas correctamente.

Ejecución interrumpida por el usuario. Finalizando el programa...
Conexión a MongoDB cerrada.


In [5]:
import pymongo
import pandas as pd

try:
    # Conectar a MongoDB
    client = pymongo.MongoClient('mongodb://localhost:27017/')  # Ajusta la URI si es necesario
    db = client['bicis_db']  # Nombre de tu base de datos
    collection = db['bicicorunha']  # Nombre de tu colección

    # Verificar la conexión
    client.admin.command('ping')  # Verifica si la conexión es exitosa
    print("Conexión a MongoDB exitosa.")

    # Obtener los datos de MongoDB
    data = list(collection.find())  # Recupera todos los documentos de la colección

    # Extraer los datos relevantes de cada estación
    stations_data = []
    for doc in data:
        # Asegúrate de que cada documento tiene una estación
        station = doc  # Cada documento es una estación

        # Extraer los datos de la estación
        extra_data = station.get('extra', {})
        
        # Crear el diccionario con los datos de la estación
        station_data = {
            'station_id': station.get('id'),
            'station_name': station.get('name'),
            'timestamp': station.get('timestamp'),
            'free_bikes': station.get('free_bikes'),
            'empty_slots': station.get('empty_slots'),
            'extra_uid': extra_data.get('uid'),
            'extra_last_updated': extra_data.get('last_updated'),
            'extra_slots': extra_data.get('slots'),
            'extra_normal_bikes': extra_data.get('normal_bikes'),
            'extra_ebikes': extra_data.get('ebikes'),
        }
        stations_data.append(station_data)

    # Crear el DataFrame
    df = pd.DataFrame(stations_data)

    print(df)
    
    # Exportar el DataFrame a un archivo CSV y Parquet
    df.to_csv('bicicorunha_stations.csv', index=False)
    df.to_parquet('bicicorunha_stations.parquet', index=False)

    print("Datos exportados correctamente.")

except pymongo.errors.ServerSelectionTimeoutError as e:
    print(f"Error al conectar a MongoDB: {e}")
except Exception as e:
    print(f"Se produjo un error: {e}")


Conexión a MongoDB exitosa.
                           station_id           station_name  \
0                                None                   None   
1                                None                   None   
2                                None                   None   
3                                None                   None   
4                                None                   None   
..                                ...                    ...   
135  f04ee4ffca2a6f3daacba88a9eee8487              Millenium   
136  f702a7b2b1631ace3781c405acb99b70       Praza de Ourense   
137  f86954a7b51703f735bce5b549cb4e16        Parque del Agra   
138  fb0d3f40ea7570546b171b43ccd57a2a        Rosales-Labañou   
139  fee1ae73b32ffb83e1bf3f6e83e616b7  Centro Cívico Monelos   

                       timestamp  free_bikes  empty_slots extra_uid  \
0                           None         NaN          NaN      None   
1                           None         NaN          NaN    